# Modulos

In [1]:
#importamos módulos
import pandas as pd
from calendar import isleap

# Funciones

## Lectura de datos

Faltan las validaciones de fechas

In [2]:
def leer_datos():
    cantidad_inicial=input('Cantidad inicial')
    cantidad_inicial=float(cantidad_inicial)
    dia_i=input('Dia incial ')
    mes_i=input('Mes incial ')
    anio_i=input('Año incial ')
    dia_f=input('Dia final ')
    mes_f=input('Mes final ')
    anio_f=input('Anio incial ')
    return cantidad_inicial,dia_i,mes_i,anio_i,dia_f,mes_f,anio_f
    '''print('¿Deseas modificar algún dato?')
    modificar=input().lower()
    if modificar=='sí' or modificar=='si':
        print('¿Qué dato deseas modificar?')
        print('Cantidad inicial 0 ')
        print('Dia incial 1 ')
        print('Mes incial 2 ')
        print('Año incial 4 ')
        print('Dia final 5 ')
        print('Mes final 6 ')
        print('Anio incial 7 ')'''

## Conversión de suma asegurada a Udis

In [12]:
def cambio_udis(anio_i,mes_i,dia_i,anio_f,mes_f,dia_f,udis,cantidad_inicial):
    
    fecha_udi_i=anio_i+'-'+mes_i+'-'+dia_i
    fecha_udi_f=anio_f+'-'+mes_f+'-'+dia_f
    print(f'Valor UDIS en {fecha_udi_i}:',float(udis[udis.Fecha==fecha_udi_i]['Valor de UDIS']))
    print(f'Valor UDIS en {fecha_udi_f}:',float(udis[udis.Fecha==fecha_udi_f]['Valor de UDIS']))
    # Suma Asegurada en pesos Final (la que se suma) 
    SA_pesos_f = (float(udis[udis.Fecha==fecha_udi_f]['Valor de UDIS'])*cantidad_inicial)/float(udis[udis.Fecha==fecha_udi_i]['Valor de UDIS'])
    # Suma Asegurada Inicial (con la que se sacan los moratorios)
    SA_udis_i = (float(cantidad_inicial)/float(udis[udis.Fecha==fecha_udi_i]['Valor de UDIS']))
    valor_udis_f = float(udis[udis.Fecha==fecha_udi_f]['Valor de UDIS'])
    return SA_pesos_f, SA_udis_i, valor_udis_f

## Días trasncurridos en cada mes

In [4]:
def get_days(aux, dia_i, dia_f):
    dc_dias= {1: 31, 2:28, 3:31, 4: 30, 5:31, 6:30,
              7: 31, 8: 31, 9:30, 10: 31, 11:30, 12:31}
    contador=0
    ls_dias= []
    size=len(aux) -1
    for index, row in aux.iterrows():
        #print(dc_dias[row['Título'].month])
        if(contador==0):
            dias= dc_dias[row['Título'].month]-dia_i +1
        elif(contador==size):
            dias= dia_f
        else:
            dias= dc_dias[row['Título'].month]
        contador+=1
        if(row['Título'].month==2 and isleap(row['Título'].year)):
            dias+=1
        ls_dias.append(dias)
    aux["Dias"]= ls_dias
    return aux

## Calculo intereses moratorios

In [5]:
def calculateInterests(aux,udis_SA,dia_i,dia_f):
    tabla=get_days(aux, int(dia_i),int(dia_f))
    tabla["Tasa anual"]= tabla["CCP_UDIS"].map(lambda x: (x*1.25)/100)
    tabla["Tasa diaria"]= tabla["Tasa anual"].map(lambda x: x/365)
    ls_interes= []
    for _,row in tabla.iterrows():
        tasa_diaria= row["Tasa diaria"]
        dias= row["Dias"]
        ls_interes.append(udis_SA*dias*tasa_diaria)
    tabla["Intereses moratorios"]= ls_interes
    
    return tabla

# Calculadora de indemnización

In [14]:
def calculadora():
    #importar datos
    data_cap=pd.read_excel('Costo_Captacion_mensual.xlsx')
    udis=pd.read_excel('UDIS_Diario.xlsx')
    
    cantidad_inicial,dia_i,mes_i,anio_i,dia_f,mes_f,anio_f=leer_datos()
    fecha_i=anio_i+'-'+mes_i+'-'+dia_i
    fecha_f=anio_f+'-'+mes_f+'-'+dia_f
    
    SA_pesos_f, SA_udis_i, valor_udis_f = cambio_udis(anio_i,mes_i,dia_i,anio_f,mes_f,dia_f,udis,cantidad_inicial)
    fecha_aux = anio_i+'-'+mes_i
    aux=data_cap[(data_cap.Título >=fecha_aux) & (data_cap.Título < fecha_f)].copy()
    tabla=calculateInterests(aux,SA_udis_i,dia_i,dia_f)
    intereses= tabla["Intereses moratorios"].sum()*valor_udis_f 
    rendimiento_udis= SA_pesos_f-cantidad_inicial
    Total=intereses+ SA_pesos_f
    print('-------------------------------------------------------')
    print("Segun lo previsto en la ley tal... los pagos correspondientes son: ")
    print(f"a) Deuda inicial = ${cantidad_inicial}")
    print(f"b) Rendimiento de la Udis = ${rendimiento_udis}")
    print(f"c) Intereses moratorios= ${intereses}")
    print(f"Total =${Total}")
    

In [6]:
#importar datos
data_cap=pd.read_excel('Costo_Captacion_mensual.xlsx')
udis=pd.read_excel('UDIS_Diario.xlsx')

# Ejemplo de uso

In [15]:
calculadora()

Cantidad inicial 4000000
Dia incial  20
Mes incial  02
Año incial  2018
Dia final  17
Mes final  07
Anio incial  2022


Valor UDIS en 2018-02-20: 5.993613
Valor UDIS en 2022-07-17: 7.399308
-------------------------------------------------------
Segun lo previsto en la ley tal... los pagos correspondientes son: 
a) Deuda inicial = $4000000.0
b) Rendimiento de la Udis = $938128.6379350824
c) Intereses moratorios= $1209425.4958677483
Total =$6147554.133802831
